In [3]:
import bisect
from pprint import pprint

In [4]:
# Hash to a potential node ID
finger = lambda n, m, i: (n + 2**i) % 2**m
# Round our potential node ID to the next real node
ring = lambda vals, n: vals[bisect.bisect(vals, n) % len(vals)]
# Find the whole table's worth of neighbors
ftable = lambda n, m, vals: [ring(vals, x) for x in [finger(n, m, i) for i in range(m)]]

In [11]:
# Node IDs on the ring
intervals = [45, 32, 132, 234, 99, 199]
intervals.sort()
# Size of finger table
m = 8
# Print out finger table for all entries
tables = [(n, ftable(n, m, intervals)) for n in intervals]
pprint(tables)

[(32, [45, 45, 45, 45, 99, 99, 99, 199]),
 (45, [99, 99, 99, 99, 99, 99, 132, 199]),
 (99, [132, 132, 132, 132, 132, 132, 199, 234]),
 (132, [199, 199, 199, 199, 199, 199, 199, 32]),
 (199, [234, 234, 234, 234, 234, 234, 32, 99]),
 (234, [32, 32, 32, 32, 32, 32, 45, 132])]


In [51]:
def has_neighbor(target, neighbors, m):
    """Check finger tables for inclusion of neighbor."""
    tables = [(n, ftable(n, m, neighbors)) for n in neighbors]
    return [ n for n, table in tables if target in table]            

In [53]:
has_45 = has_neighbor(target=45, neighbors=intervals, m=8)
print(has_45)

[32, 234]


In [60]:
def compute_hops(chord, source, key, m):
    """Compute the path taken in a Chord DHT from source to <key>."""
    # Make a copy and sort it 
    nodes = list(chord)
    nodes.sort()  # Make sure we're sorted
    # Find where the key's stored
    dest = ring(nodes, key)
    # Start at the beginning
    n = source
    # Track where we go
    hops = []
    while n != dest:
        hops.append(n)
        # Compute the finger table at this node
        table = ftable(n, m, nodes)
        
        if dest in table:
            # Go to key
            hops.append(dest)
            break
            
        closer = next((x for x in table if x < key), False)
        if closer:
            # Go to next closest neighbor
            n = closer
        else:
            # Go to successor
            n = nodes[(nodes.index(n) + 1) % len(nodes)]
            
    return hops

In [62]:
print(compute_hops([1, 12, 123, 234, 345, 456, 501], 234, 10, 9))

[234, 345, 456, 12]
